# PlayerUnknown's Battlegrounds Behavioral Analysis
### Xander Hieken

Most of the cells in this notebook were reused many times throughout the project to make quick comparisons and explore the data.

In [ ]:
from pyspark.sql.functions import abs
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
from pyspark.sql.types import DoubleType
from pyspark.mllib.stat import Statistics
from pyspark.ml.stat import Correlation
import pyarrow
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.pyplot import imread
import matplotlib.markers
from matplotlib.collections import PatchCollection
import numpy as np
import pandas as pd
import seaborn as sns
import re
import os
import PIL
import mplcyberpunk

In [ ]:
#Spark Configuration
warehouse_dir = 'DataWarehouse/'
spark = SparkSession.builder \
    .appName("PUBG Data Warehouse") \
    .config("spark.sql.warehouse.dir", warehouse_dir) \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "28g") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseCompressedOops") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

***
### The following cell only needs to be run once to convert the CSV files into Parquet files

In [ ]:
# files_to_add is a dictionary of file paths to be added to the data warehouse
files_to_add = ['PUBGkills.csv', 'PUBGstats.csv']

for i in files_to_add:
    df = spark.read.load(i, format='csv', sep=',', inferSchema=True, header=True) # loads 2017 files
    isolateTable = re.search('(?<=PUBG)(.*).csv', i) # isolates the string I want to use as the table name
    tableName = isolateTable.group(1) # sets tableName to whatever is between 'PUBG' and '.csv' in the file path
    df.write.saveAsTable(tableName, mode = 'overwrite') # saves the new tables

***
### Creating temporary views to work with in Spark SQL

In [ ]:
#Over the course of this project, I kept coming back and adjusting things as needed, so this is not

dfKills = spark.read.load("DataWarehouse/kills")
dfKills.select('killed_by','victim_name','victim_placement','map','victim_position_x','victim_position_y','match_id'
              ).createOrReplaceTempView("kills")

dfStats = spark.read.load("DataWarehouse/stats")
dfStats.select('player_name', 'player_kills', 'player_assists', 'player_dmg', 'player_survive_time',
               'team_placement', 'player_dbno','player_dist_ride', 'player_dist_walk','game_size', 'party_size',
               'date', 'match_id').createOrReplaceTempView('stats')

spark.sql("SELECT * FROM stats LEFT JOIN kills ON kills.match_id = stats.match_id "
          "AND kills.victim_name = stats.player_name").createOrReplaceTempView('combined')

spark.catalog.dropTempView("kills")
spark.catalog.dropTempView("stats")

spark.sql("SELECT * "
          "FROM combined "
          "WHERE map = 'ERANGEL' "
          "AND victim_position_x <= 819200 "   
          "AND victim_position_y <= 819200 "   
          "AND victim_position_x >= 0 "
          "AND victim_position_y >= 0 "
          "AND player_dist_walk + player_dist_ride > 10 "
          "AND team_placement > 0 "             
          "AND player_survive_time <= 2400 "    
          "AND player_survive_time >= 120"
         ).createOrReplaceTempView('combined')

In [ ]:
#Takes the combined data and splits it into separate views for each game mode

spark.sql("SELECT * FROM combined WHERE party_size = 1").createOrReplaceTempView('solos')
spark.sql("SELECT * FROM combined WHERE party_size = 2").createOrReplaceTempView('duos')
spark.sql("SELECT * FROM combined WHERE party_size = 4").createOrReplaceTempView('squads')
spark.catalog.dropTempView('combined')

***
# Generating Kill Maps

In [ ]:
#Creating a minimal dataframe to plot all the kills/deaths on the Erangel map
erDF = spark.sql("SELECT victim_position_x, victim_position_y FROM solos")

#Adjusting the coordinates to the resolution of the map image
erDF = erDF.withColumn('victim_position_x', erDF.victim_position_x * 8192 / 813400) \
           .withColumn('victim_position_y', erDF.victim_position_y * 8192 / 813400) \

vx = np.array(erDF.select('victim_position_x').collect())
vy = np.array(erDF.select('victim_position_y').collect())

In [ ]:
#Plotting all kill/death locations on the map of Erangel
bg = imread('pubg-match-deaths/erangel2.jpeg')

fig, ax = plt.subplots(figsize=(82,82), dpi=100)
ax.set_xlim(0, 8192); ax.set_ylim(0, 8192)
ax.imshow(bg)

plt.scatter(x=vx, y=vy, c='crimson', alpha=0.2, s=.2, marker='.')
plt.scatter(x=vx1, y=vy1, c='crimson', alpha=0.2, s=.2, marker='.')
plt.scatter(x=vx2, y=vy2, c='crimson', alpha=.2, s=.2, marker='.')

plt.gca().invert_yaxis()
plt.show()

***
# Getting counts of players that barely move

In [ ]:
spark.sql("SELECT count(*) "
          "FROM solos "
          "WHERE player_dist_walk + player_dist_ride = 0 "
         ).show()

In [ ]:
spark.sql("SELECT count(*) "
          "FROM solos "
          "WHERE player_dist_walk + player_dist_ride <= 10 "
         ).show()

In [ ]:
spark.sql("SELECT count(*) "
          "FROM solos "
          "WHERE player_dist_walk + player_dist_ride <= 100 "
         ).show()

In [ ]:
spark.sql("SELECT count(*) "
          "FROM solos "
          "WHERE player_dist_walk + player_dist_ride <= 1000 "
         ).show()

***
# Plotting distance travelled 

In [ ]:
soloDF = np.array(spark.sql("SELECT player_dist_walk + player_dist_ride AS distance "
                            "FROM solos "
                            "WHERE team_placement <= 5 "
                           ).collect())

duoDF = np.array(spark.sql("SELECT player_dist_walk + player_dist_ride AS distance "
                            "FROM duos "
                            "WHERE team_placement <= 5 "
                           ).collect())

squadDF = np.array(spark.sql("SELECT player_dist_walk + player_dist_ride AS distance "
                            "FROM squads "
                            "WHERE team_placement <= 5 "
                           ).collect())

In [ ]:
plt.style.use('cyberpunk')  
plt.rcParams['axes.axisbelow'] = True
fig, ax = plt.subplots(figsize=(32,18))
ax.set_xlim(0, 15000); ax.set_ylim(0, 21000)

plt.xticks(np.arange(0, 15001, 500))
plt.yticks(np.arange(0, 21001, 700))

plt.hist(squadDF, bins=50000, alpha=1, color='#FE53BB', label='Squads')
plt.hist(duoDF, bins=50000, alpha=.8, color='#00ff41', label='Duos')
plt.hist(soloDF, bins=50000, alpha=.8, color='#08F7FE', label='Solos')

plt.xlabel('Distance Traveled')
plt.ylabel('Count')
plt.title('Distance Traveled for Top 5 Placement')

plt.legend(loc='upper right', frameon=False, fontsize='xx-large')

#mplcyberpunk.add_glow_effects()
plt.show();

***
# Plotting deaths over time

In [ ]:
squadDF = np.array(spark.sql("SELECT player_survive_time AS time FROM squads "
                             "WHERE player_survive_time >=720").collect())

duoDF = np.array(spark.sql("SELECT player_survive_time AS time FROM duos "
                             "WHERE player_survive_time >=720").collect())

soloDF = np.array(spark.sql("SELECT player_survive_time AS time FROM solos "
                             "WHERE player_survive_time >=720").collect())

In [ ]:
plt.style.use('cyberpunk')  
plt.rcParams['axes.axisbelow'] = True
fig, ax = plt.subplots(figsize=(32,18))
ax.set_xlim(0, 2220); ax.set_ylim(0, 480)

plt.xticks(np.arange(0, 2221, 60)) 
plt.yticks(np.arange(0, 481, 20))

#Vertical lines to show when the Bluezone reaches the circle and stops moving
#plt.axvline(720, 0, 1, c='crimson', alpha=1) 
plt.axvline(1060, 0, 1, c='crimson', alpha=1)
plt.axvline(1300, 0, 1, c='crimson', alpha=1)
plt.axvline(1480, 0, 1, c='crimson', alpha=1)
plt.axvline(1640, 0, 1, c='crimson', alpha=1)
plt.axvline(1760, 0, 1, c='crimson', alpha=1)
plt.axvline(1920, 0, 1, c='crimson', alpha=1)
plt.axvline(2010, 0, 1, c='crimson', alpha=1)

#Vertical lines to show when the Bluezone starts shrinking
plt.axvline(420, 0, 1, c='dodgerblue', alpha=1)  
plt.axvline(920, 0, 1, c='dodgerblue', alpha=1)
plt.axvline(1210, 0, 1, c='dodgerblue', alpha=1)
plt.axvline(1420, 0, 1, c='dodgerblue', alpha=1)
plt.axvline(1600, 0, 1, c='dodgerblue', alpha=1)
plt.axvline(1730, 0, 1, c='dodgerblue', alpha=1)
plt.axvline(1850, 0, 1, c='dodgerblue', alpha=1)
plt.axvline(1980, 0, 1, c='dodgerblue', alpha=1)

#plt.axvline(120, 0, 1, c='F5D300', alpha=0.6) #1st circle shown
#plt.axvline(2220, 0, 1, c='F5D300', alpha=0.6) #No more safezone


plt.hist(squadDF, bins=100000, alpha=1, color='#FE53BB', label='Squads')
plt.hist(duoDF, bins=100000, alpha=.8, color='#00ff41', label='Duos')
plt.hist(soloDF, bins=100000, alpha=.8, color='#08F7FE', label='Solos')

plt.xlabel('Time of Death (seconds from start of the game)') 
plt.ylabel('Count')
plt.title('Deaths Over Time (Starting After First Circle)')
plt.legend(loc='upper left', frameon=False, fontsize='xx-large')
mplcyberpunk.add_glow_effects()
plt.show();

***
# Random SQL Queries

In [ ]:
spark.sql("SELECT killed_by, count(*) AS count "
          "FROM squads "
          "GROUP BY killed_by "
          "ORDER BY count DESC").show(50)